## Tokenized news를 TFIDF로 벡터화

In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
import numpy as np
import pandas as pd
import json
import os
from collections import defaultdict
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale
from sklearn.metrics import silhouette_samples, silhouette_score

### news 데이터 취합

In [14]:
df_tmp = pd.read_csv('../data/tokenized_news/tokenized_news')

In [44]:
tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=0.15)

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(df_tmp['text'])

In [45]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 
# 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [46]:
df_tmp['cluster_label'] = cluster_label
df_tmp

,Unnamed: 0,date,title,text,media,cluster_label
0,0,2020-01-01,"일과, 경계, 노동","비정규직, 표현, 정부, 신규, 일자리, 비정규직, 정규직, 전환, 주력, 사이, ...",경향신문,2
1,1,2020-01-01,"일자리, 일거리, 정치, 노동","일자리, 일거리, 노동, 미래, 현실, 변화, 사회, 준비, 짐작, 노동, 미래, ...",경향신문,2
2,2,2020-01-01,"기계, 정확도, 인간, 부품, 육성, 공장, 노동","일자리, 일거리, 학습, 데이터, 가공, 기업, 우드, 웍스, 교육장, 참관, 소화...",경향신문,2
3,3,2020-01-01,"지혜, 지훈, 부탁, 인포그래픽","어르신, 계속, 나이, 새해, 크게, 정부, 정책, 발표, 혜택, 저출산, 고령, ...",경향신문,2
4,4,2020-01-01,"리포트, 함부로, 판단, 간다","새천년, 시작, 세상, 년대, 사회, 변화, 대가, 밀레니엄, 베이비, 세대, 분류...",국민일보,2
...,...,...,...,...,...,...
29036,29036,2020-08-31,"단계, 무급, 휴가, 요구, 코로나, 유행","직장인, 수도권, 사회, 거리, 두기, 단계, 시행, 일간, 무급, 휴가, 사용, ...",한국일보,1
29037,29037,2020-08-31,"조치, 환자, 비율, 육박, 거리, 두기, 단계, 효과","서울, 전역, 강화, 사회, 거리, 두기, 단계, 시행, 전문가, 방역, 효과, 의...",한국일보,1
29038,29038,2020-08-31,"내일, 서울, 시내버스, 감축, 주말, 수준","수도권, 사회, 거리, 두기, 단계, 시행, 기간, 서울, 시내버스, 야간, 운행,...",한국일보,2
29039,29039,2020-08-31,"집단, 휴업, 전공, 정부, 대강, 대치, 의료, 공백, 장기, 불가피","무기한, 집단, 어가, 전공, 인턴, 레지던트, 정부, 갈등, 벼랑, 의협, 의회,...",한국일보,2


In [47]:
tmp_df = df_tmp

In [48]:
tmp_df[tmp_df['cluster_label']==0].sort_values(by='title')

,Unnamed: 0,date,title,text,media,cluster_label
11175,11175,2020-04-07,"가구, 만원, 만원, 핫이슈, 재난, 원금","총선, 여야, 보편, 지원, 총선, 의식, 풀기, 공약, 비판, 민주당, 소득, 지...",동아일보,0
26079,26079,2020-08-06,"가구, 주택, 보편화, 시대, 정책, 실현, 가능성, 의문","부동산대책, 논란, 확산, 김태년, 추가, 대책, 마련, 김진애, 조합, 반발, 윤...",세계일보,0
17656,17656,2020-05-30,"가기, 가기, 통합, 보수, 유튜버, 딜레마","총선, 패배, 상실, 표, 출하, 창구, 우파, 결집, 소냐, 보수, 유튜브, 채널...",국민일보,0
12126,12126,2020-04-14,"가능, 샴페인, 먼저, 역대, 선거, 심판","유시민, 여권, 관계자, 압승, 분위기, 자만, 상승세, 찬물, 우려, 새누리당, ...",한국일보,0
24981,24981,2020-07-28,"가닥, 임대차, 법, 인상, 률, 범위, 지자체, 결정","정부, 여당, 추진, 중인, 임대차, 법, 전월세, 고제, 전월세, 계약, 갱신, ...",한국일보,0
...,...,...,...,...,...,...
27127,27127,2020-08-14,"흥행, 부진, 수해, 겹, 민주당, 전당대회, 외환","코로나, 수해, 겹, 치면, 일로, 예정, 민주당, 전당대회, 흥행, 참패, 위기,...",한겨레,0
12891,12891,2020-04-21,"힘자랑, 개헌, 국보법, 슈퍼, 여당","서울, 신문, 우리당, 실패, 되풀이, 공감, 수로, 비판, 사전, 차단, 진보, ...",서울신문,0
26218,26218,2020-08-07,NaN,"일자, 주택, 의원, 김회재, 송언석, 이달곤, 공통점, 지역구, 채도, 기사, 민...",서울신문,0
26910,26910,2020-08-13,NaN,"지난, 일자, 게재, 사설, 남인순, 의원, 조사, 검찰, 대처, 이유, 의원, 애...",서울신문,0


In [49]:
tmp_df[tmp_df['cluster_label']==1].sort_values(by='title')

,Unnamed: 0,date,title,text,media,cluster_label
12155,12155,2020-04-15,"가격, 리자도, 투표, 의심, 증상, 오후, 가능","코로나, 진자, 접촉, 해외, 입국, 자가, 격리, 중인, 유권자, 증상, 한해, ...",국민일보,1
16638,16638,2020-05-22,"가계, 소득, 만원, 코로나, 소비, 허울, 흑자","올해, 분기, 가계, 살림, 평균, 만원, 흑자, 실속, 흑자, 신종, 코로나바이러...",국민일보,1
3809,3809,2020-02-05,"가래, 번, 확진, 차례, 병원, 체크","우한, 폐렴, 확산, 태국, 가족, 여행, 여성, 일간, 무방비, 노출, 중국, 폐...",조선일보,1
5137,5137,2020-02-17,"가슴, 통증, 응급실, 번, 의료, 환자, 접촉","서울, 신문, 일주일, 기침, 남성, 응급실, 가기, 동네, 병원, 진료, 진료, ...",서울신문,1
7152,7152,2020-03-04,"가야, 방촌, 기초, 수급, 코로나","코로나, 확산, 가운데, 병원, 보건소, 의료, 시스템, 감염병, 치료, 집중, 이...",한겨레,1
...,...,...,...,...,...,...
15448,15448,2020-05-12,"휴업, 고액, 임대료, 클럽","지난, 황금, 연휴, 영업, 재개, 홍대, 강남, 이태원, 클럽, 주일, 다시, 휴...",중앙일보,1
14580,14580,2020-05-05,"흉터, 고글, 자국, 엄, 대구, 간호사","얼굴, 눌렸, 엄마, 고글, 환자, 병실, 채현지, 간호사, 함빡, 영상통화, 화면...",중앙일보,1
9384,9384,2020-03-23,"흡연, 취약, 코로나, 악화, 위험","코로나, 팬데믹, 흡연, 코로나, 바이러스, 감염증, 흡연, 증상, 악화, 연구, ...",조선일보,1
8573,8573,2020-03-17,"희생, 협조, 코로나, 대구, 공포","의료, 라파엘, 웰빙, 센터, 의료, 응원, 내용, 편지, 전시, 연필, 지우개, ...",경향신문,1


In [50]:
tmp_df[tmp_df['cluster_label']==2].sort_values(by='title')

,Unnamed: 0,date,title,text,media,cluster_label
26444,26444,2020-08-10,"가게, 농작물, 생계, 수단, 폭우, 복구, 엄두, 못내","섬진강, 피해, 현장, 장터, 쑥대밭, 상인, 구례군, 제방, 유실, 침수, 피해,...",문화일보,2
12120,12120,2020-04-14,"가게, 시장, 불황, 처음","코로나, 직격탄, 남대문시장, 명동, 서교동, 상권, 르포, 보증금, 카드, 자영,...",한국일보,2
15120,15120,2020-05-09,"가격, 전량, 수입, 전기요금, 인상, 가능성","전력, 수급, 계획, 주요, 내용, 전망, 친환경, 발전, 전환, 가속, 청사진, ...",세계일보,2
22805,22805,2020-07-10,"가계, 대출","신용, 대출, 주택, 추정, 기업, 대출, 조원, 급감, 두, 진정, 가계, 대출,...",세계일보,2
24051,24051,2020-07-21,"가계, 한국, 대비, 세계, 최대","분기, 기준, 주요, 개국, 코로나, 영향, 오름, 폭도, 세계, 한국, 가계, 적...",세계일보,2
...,...,...,...,...,...,...
23639,23639,2020-07-17,NaN,"소리, 고층, 아파트, 서울, 수색, 뉴타운, 빌딩, 상암, 사이, 은평구, 수색동...",서울신문,2
24184,24184,2020-07-22,NaN,"여과, 밀폐, 안해, 깔따구, 침입, 활성탄, 청소, 벌레, 지방직, 순환, 근무,...",서울신문,2
25790,25790,2020-08-04,NaN,"어제, 평택, 가평, 사망, 실종, 지난, 수도권, 중부지방, 집중호우, 피해, 경...",조선일보,2
26226,26226,2020-08-07,NaN,"법무, 오늘, 고위, 간부, 인사, 추미애, 사법연수원, 법무부, 장관, 검찰, 고...",서울신문,2


In [51]:
tmp_df[tmp_df['cluster_label']==3].sort_values(by='title')

,Unnamed: 0,date,title,text,media,cluster_label


In [52]:
tmp_df[tmp_df['cluster_label']==4].sort_values(by='title')

,Unnamed: 0,date,title,text,media,cluster_label


In [53]:
# 3개의 집합으로 군집화 (뉴스)
km_cluster = KMeans(n_clusters=3, init='k-means++', max_iter=10000)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
tmp_df['cluster_label'] = cluster_label
tmp_df.sort_values(by='cluster_label')

,Unnamed: 0,date,title,text,media,cluster_label
0,0,2020-01-01,"일과, 경계, 노동","비정규직, 표현, 정부, 신규, 일자리, 비정규직, 정규직, 전환, 주력, 사이, ...",경향신문,0
17895,17895,2020-06-02,"대통령, 트럼프, 통화, 초청","문재인, 대통령, 도널드, 트럼프, 미국, 대통령, 미국, 주요, 개국, 정상, 회...",경향신문,0
17894,17894,2020-06-02,"인종차별, 항의, 시위, 도시, 확산, 트럼프, 테러, 조직, 지정","흑인, 남성, 조지, 플로이드, 사망, 촉발, 미국, 인종차별, 항의, 시위, 지난...",경향신문,0
17892,17892,2020-06-02,"대통령, 초청, 감사","문재인, 대통령, 도널드, 트럼프, 미국, 대통령, 미국, 주요, 개국, 정상, 회...",경향신문,0
17890,17890,2020-06-02,"지속, 가능, 일자리","정부, 디지털, 뉴딜, 그린, 뉴딜, 추진, 고용, 산업, 정책, 한국판, 뉴딜, ...",경향신문,0
...,...,...,...,...,...,...
6466,6466,2020-02-27,"사진, 환자, 무증상, 폐렴","중앙, 상위, 증, 자가, 치료, 제언, 지역, 사회, 감염, 시작, 번, 환자, ...",한국일보,2
6465,6465,2020-02-27,"대구, 시청, 직원, 무더기, 자가, 격리, 청와대, 긴장","대통령, 참석, 대책, 회의, 환자, 밀접, 접촉, 경제, 부시장, 배석, 음성, ...",한국일보,2
6464,6464,2020-02-27,"모금, 손편지, 응원","방역, 최전선, 분, 보탬, 경기, 성남시, 거주, 이모, 지난, 사회, 관계, 서...",한국일보,2
6462,6462,2020-02-27,"박능후, 국내, 확산, 원인, 중국, 한국인, 발언, 논란","법사, 위서, 차례, 거듭, 확인, 주무, 장관, 신중, 지적, 박능후, 보건복지부...",한국일보,2


In [54]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['title']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [55]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
#         print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')

In [56]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=tmp_df,\
                                  feature_names=feature_names, clusters_num=3, top_n_features=30)
# top_features : 중심과 가까운 값
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['정부', '대통령', '미국', '한국', '금지', '코로나', '서울', '배포', '무단', '장관', '시장', '대한', '청와대', '경제', '지원', '지난해', '상황', '지역', '지난', '문제', '정책', '세계', '관계자', '때문', '국민', '발표', '대책', '회의', '중앙', '사실']
####### Cluster 1
Top features: ['민주당', '의원', '통합', '국회', '대표', '위원장', '미래', '국민', '위원회', '대통령', '서울', '정부', '정책', '한국', '지역', '비판', '문제', '라며', '회의', '대한', '대책', '문재인', '금지', '장관', '배포', '무단', '입장', '지난', '청와대', '코로나']
####### Cluster 2
Top features: ['감염', '진자', '코로나', '방역', '신종', '검사', '확산', '지역', '당국', '서울', '코로나바이러스', '사회', '정부', '지난', '확인', '감염증', '관리', '중앙', '조치', '상황', '사람', '조사', '금지', '추가', '전국', '우려', '대책', '오후', '지난달', '기준']


- Cluster 0 : 정치
- Cluster 1 : 사회
- Cluster 2 : 경제

In [69]:
score_samples = silhouette_samples(feature_vect, tmp_df['cluster_label'])

In [70]:
tmp_df['silhouette_coeff'] = score_samples

In [71]:
# 모든 데이터의 평균 실루엣 계수를 구함
average_score = silhouette_score(feature_vect, tmp_df['cluster_label'])

In [72]:
print('News text Silhouette Analysis Score:{0:.3f}'.format(average_score))
tmp_df['silhouette_coeff']

News text Silhouette Analysis Score:0.062


0        0.023981
1        0.017044
2        0.011661
3        0.025571
4        0.012135
           ...   
29036    0.099936
29037    0.165012
29038   -0.017985
29039    0.002235
29040   -0.009447
Name: silhouette_coeff, Length: 29041, dtype: float64